In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

## **Which GPU we are using**

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14894475721216849806, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5539015173216233833
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2191376810159381175
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14648777152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9406903633940455996
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [32]:
tf.__version__  #tensorflow version

'2.2.0'

In [1]:
cat /proc/meminfo # to check RAM info 

MemTotal:       13333556 kB
MemFree:        10443040 kB
MemAvailable:   12434480 kB
Buffers:           75372 kB
Cached:          2071700 kB
SwapCached:            0 kB
Active:           749992 kB
Inactive:        1847436 kB
Active(anon):     428276 kB
Inactive(anon):      336 kB
Active(file):     321716 kB
Inactive(file):  1847100 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               568 kB
Writeback:             0 kB
AnonPages:        450392 kB
Mapped:           246404 kB
Shmem:               952 kB
Slab:             167672 kB
SReclaimable:     127040 kB
SUnreclaim:        40632 kB
KernelStack:        4452 kB
PageTables:         5940 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666776 kB
Committed_AS:    2861996 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:              920 kB
AnonHugePages:   

In [2]:
cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.140
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa itlb_multi

In [3]:
pip install keras

In [0]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [10]:
dataset= pd.read_csv('/content/drive/My Drive/Fake-News Classifier/train.csv') #importing file from drive 
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [16]:
# checking null values in %
tab_info= pd.DataFrame(dataset.dtypes).T.rename(index= {0: 'column type'})
#print(tab_info)
tab_info= tab_info.append(pd.DataFrame(dataset.isnull().sum()).T.rename(index= {0:'null values'}))
tab_info= tab_info.append(pd.DataFrame((dataset.isnull().sum()/dataset.shape[0])*100).T.rename(index= {0:'null values in %'}))
tab_info

,id,title,author,text,label
column type,int64,object,object,object,int64
null values,0,558,1957,39,0
null values in %,0,2.68269,9.40865,0.1875,0


In [17]:
dataset= dataset.dropna()
dataset.shape

(18285, 5)

In [18]:
messages= dataset.copy()
messages= messages.reset_index(drop=True)
messages.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [19]:
messages.title[1:5]

1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [23]:
# classification will be based on title
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
corpus= []
ps= PorterStemmer()
for i in range(0,len(messages)):
    review= re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review= review.lower()
    review= review.split()
    review= [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review= ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
print(len(corpus)) # List conatining sentences
corpus[:10]

18285


['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [28]:
corpus_df= pd.Series(corpus) # converting into series
print(corpus_df.shape)
corpus_df[:10]

(18285,)


0    hous dem aid even see comey letter jason chaff...
1      flynn hillari clinton big woman campu breitbart
2                                 truth might get fire
3             civilian kill singl us airstrik identifi
4    iranian woman jail fiction unpublish stori wom...
5    jacki mason hollywood would love trump bomb no...
6    beno hamon win french socialist parti presiden...
7    back channel plan ukrain russia courtesi trump...
8    obama organ action partner soro link indivis d...
9        bbc comedi sketch real housew isi caus outrag
dtype: object

In [0]:
# Target variable
y= dataset.label

In [0]:

### Vocabulary size
voc_size=5000

In [37]:
onehot_repr= [one_hot(words, voc_size) for words in corpus]
print(type(onehot_repr)) #list type
onehot_repr[:20]  # will fetch index for each word fron vocablary size. below fetched first 20 

<class 'list'>


[[3455, 685, 2824, 1409, 1980, 3894, 482, 3076, 2445, 833],
 [4479, 1980, 3442, 3554, 1747, 4854, 3646],
 [4645, 1519, 2859, 2197],
 [3728, 2239, 1726, 1425, 1411, 2286],
 [3931, 1747, 1952, 2357, 1108, 3688, 1747, 729, 3257, 3236],
 [1697,
  1127,
  3942,
  4875,
  637,
  2062,
  4377,
  3873,
  4124,
  784,
  54,
  3206,
  4737,
  2924,
  3646],
 [4657, 2880, 1642, 4710, 2581, 4727, 2857, 4744, 3984, 2982, 4532],
 [1088, 3200, 3330, 508, 3040, 667, 2062, 4494, 3984, 2982, 4532],
 [3250, 1602, 4541, 2766, 1983, 748, 4958, 1599, 2062, 982],
 [3529, 1672, 4745, 323, 2381, 2430, 2645, 4353],
 [2975, 4340, 1451, 4337, 4754, 4813, 1682, 932, 107, 3056, 1197],
 [1425, 4121, 1980, 748, 2062, 3040],
 [4050, 3733, 2112, 3691, 2008, 4748, 4699, 4946, 2149],
 [2031, 3837, 4373, 2653, 1657, 1137, 3660, 3984, 2982, 4532],
 [1428, 2708, 900, 3087, 4368, 3984, 2982, 4532],
 [2567, 1106, 2915, 1553, 1404, 969, 2474, 1037, 1007, 1454],
 [58, 4664, 1980],
 [646, 507, 2975, 3014, 2062, 4674, 2209, 3646]

**# Embedding Representation**

In [38]:
sent_length= 20 # taking this value to maintain each sentence length to 20
embedded_docs= pad_sequences(onehot_repr, padding='pre', maxlen=sent_length) # order shud be same, use padding pre post to fill 0's
print(type(embedded_docs))
print(embedded_docs) # will convert to same length

<class 'numpy.ndarray'>
[[   0    0    0 ... 3076 2445  833]
 [   0    0    0 ... 1747 4854 3646]
 [   0    0    0 ... 1519 2859 2197]
 ...
 [   0    0    0 ... 3984 2982 4532]
 [   0    0    0 ...  613 3585 4168]
 [   0    0    0 ... 4180 3630 4992]]


In [39]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3455,
        685, 2824, 1409, 1980, 3894,  482, 3076, 2445,  833], dtype=int32)

In [40]:
## creating model
embedding_features_value= 40
model=Sequential()
model.add(Embedding(voc_size, embedding_features_value, input_length=sent_length)) #always be first layer in NN for embedding
model.add(LSTM(100)) #using 1 LSTM layer containing 100 neurons
model.add(Dense(1, activation='sigmoid')) # last layer to predict o/p 0 or 1
model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics= ['accuracy']) # which will compile with backprop. and update weights
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
dataset['label'].head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [41]:
# Making train and test data, Both has to be in array matrix to compute
X= embedded_docs
Y= dataset['label']
print(X.shape)
print(Y.shape)

(18285, 20)
(18285,)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

**Model Training**

In [52]:
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0027 - accuracy: 0.9994 - val_loss: 0.6940 - val_accuracy: 0.9077
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0032 - accuracy: 0.9990 - val_loss: 0.7131 - val_accuracy: 0.9089
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0023 - accuracy: 0.9994 - val_loss: 0.5532 - val_accuracy: 0.9072
Epoch 4/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0016 - accuracy: 0.9995 - val_loss: 0.7826 - val_accuracy: 0.9034
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0075 - accuracy: 0.9981 - val_loss: 0.6925 - val_accuracy: 0.9100
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0031 - accuracy: 0.9993 - val_loss: 0.7443 - val_accuracy: 0.9042
Epoch 7/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0013 - accuracy: 0.9998 - val_loss: 0.7962 - val_accuracy: 0.9044
Epoch

In [50]:
## Above accuracy is 1.0 so it is cleary overfit the model
pred= model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [53]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

score= accuracy_score(y_test, pred)
print(score)
print(confusion_matrix(y_test, pred))

0.912013256006628
[[3094  325]
 [ 206 2410]]


**Adding Dropout Layer to avoid overfitting**

In [0]:
from tensorflow.keras.layers import Dropout

embedding_features_value= 40
model1=Sequential()
model1.add(Embedding(voc_size, embedding_features_value, input_length= sent_length))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [66]:
model1.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0071 - accuracy: 0.9972 - val_loss: 0.5801 - val_accuracy: 0.8974
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0100 - accuracy: 0.9961 - val_loss: 0.5445 - val_accuracy: 0.9109
Epoch 3/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0057 - accuracy: 0.9978 - val_loss: 0.6629 - val_accuracy: 0.9027
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0068 - accuracy: 0.9981 - val_loss: 0.6505 - val_accuracy: 0.9100
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0066 - accuracy: 0.9984 - val_loss: 0.6616 - val_accuracy: 0.9074
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0046 - accuracy: 0.9982 - val_loss: 0.6887 - val_accuracy: 0.8969
Epoch 7/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0028 - accuracy: 0.9993 - val_loss: 0.7874 - val_accuracy: 0.9022
Epoch 

In [0]:
# Predicting on test data

pred= model1.predict_classes(X_test)

In [68]:
score= accuracy_score(y_test, pred)
print(score)
print(confusion_matrix(y_test, pred))

0.9065451532725767
[[3091  328]
 [ 236 2380]]


In [69]:
# classification report

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

